In [1]:
library(igraph)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



## 2 Let’s Help Santa!
Companies like Google and Uber have a vast amount of statistics about transportation dynamics. Santa has decided to use network theory to facilitate his gift delivery for the next christmas. When we learned about his decision, we designed this part of the project to help him. We will send him your results for this part!

### 2.1 Download the Data
Go to “Uber Movement” website and download data of Monthly Aggregate (all days), 2017 Quarter 4, for San Francisco area 1. The dataset contains pairwise traveling time statistics between most pairs of points in San Francisco area. Points on the map are represented by unique IDs. To understand the correspondence between map IDs and areas, download Geo Boundaries file from the same website 2. This file contains latitudes and longitudes of the corners of the polygons circumscribing each area. In addition, it contains one street address inside each area, referred to as DISPLAY_NAME. To be specific, if an area is represented by a polygon with 5 corners, then you have a 52 matrix of the latitudes and longitudes, each row of which represents latitude and longitude of one corner.

### 2.2 Build Your Graph

Read the dataset at hand, and build a graph in which nodes correspond to locations, and undirected weighted edges correspond to the mean traveling times between each pair of locations (only December). Add the following attributes to the vertices:
1. Display name: the street address
2. Location: mean of the coordinates of the polygon’s corners (a 2-D vector)

The graph will contain some isolated nodes (extra nodes existing in the Geo Boundaries JSON file) and a few small connected components. Remove such nodes and just keep the giant connected component of the graph. In addition, merge duplicate edges by averaging their weights. We will refer to this cleaned graph as G afterwards.

- **Question 6:** Report the number of nodes and edges in G.

In [2]:
weight_mat <- as.matrix(read.csv(file="weight_mat.csv", row.names=1, header=TRUE, sep=","))
id_addr <- read.csv(file="id_addr.csv", header=TRUE, sep=",")
id_coor <- read.csv(file="id_coor.csv", header=TRUE, sep=",")

g <- graph_from_adjacency_matrix(weight_mat, weighted=TRUE, mode="max")
g <- set.vertex.attribute(g, "addr", paste0("X", id_addr$id), as.character(id_addr$addr))
g <- set.vertex.attribute(g, "coor", paste0("X", id_coor$id), as.character(id_coor$coor))

In [3]:
is.connected(g)
vcount(g)
ecount(g)

[1] FALSE

[1] 1887

[1] 311812

In [4]:
g.components <- clusters(g)
ix <- which.max(g.components$csize) 
g.giant <- induced.subgraph(g, which(g.components$membership == ix))

In [5]:
is.connected(g.giant)
vcount(g.giant)
ecount(g.giant)

[1] TRUE

[1] 1880

[1] 311802

### 2.3 Traveling Salesman Problem

- **Question 7:** Build a minimum spanning tree (MST) of graph G. Report the street addresses of the two endpoints of a few edges. Are the results intuitive?

In [6]:
g.mst <- mst(g.giant)

In [7]:
for (i in 1:5) { 
    edges = sample(E(g.mst), 1)
    vertices = strsplit(as_ids(edges)[1], split='|', fixed=TRUE)[[1]]
    print(edges)
    print(get.vertex.attribute(g, 'addr', vertices))
}

+ 1/1879 edge from 73e3e24 (vertex names):
[1] X1981--X1989
[1] "600 Crest Drive, Ben Lomond"  "11200 Visitar Street, Felton"
+ 1/1879 edge from 73e3e24 (vertex names):
[1] X597--X1072
[1] "1600 Eighth Street, Northwest Berkeley, Berkeley"  
[2] "1700 Chestnut Street, Northwest Berkeley, Berkeley"
+ 1/1879 edge from 73e3e24 (vertex names):
[1] X562--X927
[1] "300 Tioga Court, Greenmeadow, Palo Alto"
[2] "800 Aspen Way, Palo Verde, Palo Alto"   
+ 1/1879 edge from 73e3e24 (vertex names):
[1] X143--X301
[1] "1300 14th Street, Junior College Neighborhood Association, Santa Rosa"
[2] "2400 Grahn Drive, SOS, Santa Rosa"                                    
+ 1/1879 edge from 73e3e24 (vertex names):
[1] X487--X1636
[1] "2300 Dover Way, Pittsburg"  "2300 Range Road, Pittsburg"


In [28]:
mst.cost = 0
for (i in 1:ecount(g.mst)) { 
    edges = E(g.mst)[i]
    vertices = strsplit(as_ids(edges)[1], split='|', fixed=TRUE)[[1]]
    dis = distances(g.mst, v=vertices[1], to=vertices[2])[1]
    mst.cost = mst.cost + dis
}
mst.cost

[1] 301761.4

- **Question 8:** Determine what percentage of triangles in the graph (sets of 3 points on the map) satisfy the triangle inequality. You do not need to inspect all triangles, you can just estimate by random sampling of 1000 triangles.

In [9]:
true_count = 0
n = 1000

for (i in 1:n) {
    points = sample(V(g.giant), 3, replace=FALSE)
    a = points[1]
    b = points[2]
    c = points[3]

    ab = distances(g.giant, v=a, to=b)[1]
    ac = distances(g.giant, v=a, to=c)[1]
    bc = distances(g.giant, v=b, to=c)[1]

    if (ab + ac >= bc && ab + bc >= ac && ac + bc >= ab) {
        true_count = true_count + 1
    }
}
percentage = true_count / n * 100
print(paste("Percentage: ", percentage, "%", sep=''))

[1] "Percentage: 100%"


Now, we want to find an approximation solution for the traveling salesman problem (TSP) on G. Apply the 2-approximate algorithm described in the class. Inspect the sequence of street addresses visited on the map and see if the results are intuitive.

- **Question 9:** Find the empirical performance of the approximate algorithm: 
$p = \frac{Approximate TSP Cost}{Optimal TSP Cost}$

In [10]:
node_list <- dfs(g.mst, root=V(g.mst)$name[1])$order

In [11]:
for (vertex in node_list) {
    print(get.vertex.attribute(g.giant, 'addr', vertex))
}

[1] "400 Northumberland Avenue, Redwood Oaks, Redwood City"
[1] "1500 Oxford Street, Palm Park, Redwood City"
[1] "100 Fifth Avenue, South Fair Oaks, Redwood City"
[1] "400 Seventh Avenue, Fair Oaks, Menlo Park"
[1] "2800 Fair Oaks Avenue, Redwood City"
[1] "700 Bay Road, Staumbaugh Heller, Redwood City"
[1] "100 Hinman Road, Redwood City"
[1] "700 Industrial Road, San Carlos"
[1] "400 Oxford Way, Belmont"
[1] "3000 Los Prados Street, East San Mateo, San Mateo"
[1] "1700 Eisenhower Street, Shoreview, San Mateo"
[1] "600 Vanessa Drive, Central San Mateo, San Mateo"
[1] "300 East 28th Avenue, South San Mateo, San Mateo"
[1] "200 Cupertino Way, South San Mateo, San Mateo"
[1] "1100 Vailwood Way, South San Mateo, San Mateo"
[1] "200 West 25th Avenue, South San Mateo, San Mateo"
[1] "2700 Campus Drive, Beresford Park, San Mateo"
[1] "1600 Parrott Drive, South San Mateo, San Mateo"
[1] "1400 Crestwood Court, South San Mateo, San Mateo"
[1] "2600 Somerset Drive, Belmont"
[1] "Cypress Ridge Ro

In [12]:
g_ <- graph.empty(0, directed=FALSE)
g_ <- add_vertices(g_, vcount(g.giant))
V(g_)$name <- V(g.giant)$name

In [13]:
index_vertex = data.frame(index=1:vcount(g_), vertex=node_list$name)

In [31]:
cost = 0
for (index in 1:vcount(g_)) {
    item = index_vertex$vertex[index_vertex$index==index]
    if (index < vcount(g_)) {
        item_ = index_vertex$vertex[index_vertex$index==index+1]
    } else {
        item_ = index_vertex$vertex[index_vertex$index==1]
    }
    g_ <- add_edges(g_, c(item, item_))
    dis = distances(g.giant, v=item, to=item_)[1]
    cost = cost + dis
} 
print(cost)

[1] 505449.7


- **Question 10:** Plot the trajectory that Santa has to travel!

In [15]:
a = c()
for (vertex in node_list) {
    a = c(a, V(g_)$name[vertex])
}
write.csv(a, file="TSP.csv", row.names=FALSE)
# plot is in python file